In [17]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

url = "http://www.cbr.ru/scripts/XML_daily.asp"

today = datetime.today()
today = today.strftime("%d/%m/%Y")

payload = {"date_req" : today}

responce = requests.get(url, params=payload)
#print(responce.text)
# print(responce.url)
xml = BeautifulSoup(responce.content, "lxml")

def get_val(ch):
	if ch == "RUR": return ("Российский рубль",1,1)
	if ch not in responce.text: return False
	a = responce.text.split(ch)[1]
	b = a.split("</Valute>")[0]
	nom = b.split("<Nominal>")[1].split("</Nominal>")[0]
	name = b.split("<Name>")[1].split("</Name>")[0]
	val = b.split("<Value>")[1].split("</Value>")[0]
	return (name,int(nom),float(val.replace(",",".")))

def converter(valute_from,valute_to, amount):
	valute_from = get_val(valute_from)
	valute_to = get_val(valute_to)
	if valute_to!=False and valute_from!=False:
		a = amount*valute_from[2]/valute_from[1]
		b = a*valute_to[1]/valute_to[2]
		return f"За {amount} {valute_from[0]}  вы получите {b} {valute_to[0]}"
	else:
		return "Валюта не найдена"


def getCourse (id):
	return xml.find("valute",  {'id': id}).value.text


print(getCourse("R01235"), "рублей за 1 доллар")
print(getCourse("R01239"), "рублей за 1 евро")
print(getCourse("R01375"), "рублей за 10 юаней")

print(converter("RUR","RUR",100))




74,7588 рублей за 1 доллар
79,6104 рублей за 1 евро
10,8409 рублей за 10 юаней
За 100 Российский рубль  вы получите 100.0 Российский рубль
